In [1]:
import kagglehub
import pandas as pd
import os
import pyGMs as gm
import numpy as np
import pandas as pd
import numpy as np
import torch
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
import argparse

In [2]:
# Download latest version
path = kagglehub.dataset_download("drgilermo/nba-players-stats")
print("Path to dataset files:", path)

Path to dataset files: C:\Users\sheik\.cache\kagglehub\datasets\drgilermo\nba-players-stats\versions\2


In [3]:
df = pd.read_csv(os.path.join(path, "Seasons_Stats.csv"), encoding='latin1')

## Data Visualization and Cleaning

In [4]:
df.head(10)

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,0,1950.0,Curly Armstrong,G-F,31.0,FTW,63.0,NaN,NaN,NaN,...,0.705,NaN,NaN,NaN,176.0,NaN,NaN,NaN,217.0,458.0
1,1,1950.0,Cliff Barker,SG,29.0,INO,49.0,NaN,NaN,NaN,...,0.708,NaN,NaN,NaN,109.0,NaN,NaN,NaN,99.0,279.0
2,2,1950.0,Leo Barnhorst,SF,25.0,CHS,67.0,NaN,NaN,NaN,...,0.698,NaN,NaN,NaN,140.0,NaN,NaN,NaN,192.0,438.0
3,3,1950.0,Ed Bartels,F,24.0,TOT,15.0,NaN,NaN,NaN,...,0.559,NaN,NaN,NaN,20.0,NaN,NaN,NaN,29.0,63.0
4,4,1950.0,Ed Bartels,F,24.0,DNN,13.0,NaN,NaN,NaN,...,0.548,NaN,NaN,NaN,20.0,NaN,NaN,NaN,27.0,59.0
5,5,1950.0,Ed Bartels,F,24.0,NYK,2.0,NaN,NaN,NaN,...,0.667,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2.0,4.0
6,6,1950.0,Ralph Beard,G,22.0,INO,60.0,NaN,NaN,NaN,...,0.762,NaN,NaN,NaN,233.0,NaN,NaN,NaN,132.0,895.0
7,7,1950.0,Gene Berce,G-F,23.0,TRI,3.0,NaN,NaN,NaN,...,0.000,NaN,NaN,NaN,2.0,NaN,NaN,NaN,6.0,10.0
8,8,1950.0,Charlie Black,F-C,28.0,TOT,65.0,NaN,NaN,NaN,...,0.651,NaN,NaN,NaN,163.0,NaN,NaN,NaN,273.0,661.0
9,9,1950.0,Charlie Black,F-C,28.0,FTW,36.0,NaN,NaN,NaN,...,0.632,NaN,NaN,NaN,75.0,NaN,NaN,NaN,140.0,382.0


In [5]:
df['Player'] = df['Player'].str.replace('*', '', regex=False)
df = df.drop(columns=["Unnamed: 0"])
df = df.drop(columns=["blanl", "blank2"])
df = df[df['Year'].notna()]
df['Year'] = df['Year'].astype(int)
df = df[df['Year'] > 1973]

In [6]:
df.head(10)

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
3851,1974,Zaid Abdul-Aziz,C,27.0,HOU,79.0,NaN,2459.0,15.9,0.516,...,0.804,259.0,664.0,923.0,166.0,80.0,104.0,NaN,227.0,865.0
3852,1974,Kareem Abdul-Jabbar,C,26.0,MIL,81.0,NaN,3548.0,24.4,0.564,...,0.702,287.0,891.0,1178.0,386.0,112.0,283.0,NaN,238.0,2191.0
3853,1974,Don Adams,SF,26.0,DET,74.0,NaN,2298.0,10.9,0.457,...,0.761,133.0,315.0,448.0,141.0,110.0,12.0,NaN,242.0,759.0
3854,1974,Rick Adelman,PG,27.0,CHI,55.0,NaN,618.0,10.0,0.447,...,0.711,16.0,53.0,69.0,56.0,36.0,1.0,NaN,63.0,182.0
3855,1974,Lucius Allen,PG,26.0,MIL,72.0,NaN,2388.0,18.8,0.536,...,0.788,89.0,202.0,291.0,374.0,137.0,22.0,NaN,215.0,1268.0
3856,1974,Tiny Archibald,PG,25.0,KCO,35.0,NaN,1272.0,18.3,0.527,...,0.820,21.0,64.0,85.0,266.0,56.0,7.0,NaN,76.0,617.0
3857,1974,Dennis Awtrey,C,25.0,CHI,68.0,NaN,756.0,11.4,0.560,...,0.574,49.0,125.0,174.0,86.0,22.0,14.0,NaN,128.0,184.0
3858,1974,Mike Bantom,PF,22.0,PHO,76.0,NaN,1982.0,11.7,0.437,...,0.662,172.0,347.0,519.0,163.0,50.0,47.0,NaN,289.0,769.0
3859,1974,Dick Barnett,SG,37.0,NYK,5.0,NaN,58.0,9.4,0.403,...,0.667,1.0,3.0,4.0,6.0,1.0,0.0,NaN,2.0,22.0
3860,1974,Jim Barnett,SG,29.0,GSW,77.0,NaN,1689.0,16.4,0.517,...,0.814,76.0,146.0,222.0,209.0,56.0,11.0,NaN,146.0,884.0


In [7]:
nan_count = np.sum(df.isnull(), axis=0)
print(nan_count)
print(df.shape)

Year         0
Player       0
Pos          0
Age          0
Tm           0
G            0
GS        2564
MP           0
PER          5
TS%         82
3PAr      1958
FTr         94
ORB%         5
DRB%         5
TRB%         5
AST%         5
STL%         5
BLK%         5
TOV%      1215
USG%      1157
OWS          0
DWS          0
WS           0
WS/48        5
OBPM         0
DBPM         0
BPM          0
VORP         0
FG           0
FGA          0
FG%         94
3P        1870
3PA       1870
3P%       5381
2P           0
2PA          0
2P%        123
eFG%        94
FT           0
FTA          0
FT%        792
ORB          0
DRB          0
TRB          0
AST          0
STL          0
BLK          0
TOV       1152
PF           0
PTS          0
dtype: int64
(20797, 50)


## Label Creation and Training Data

In [8]:
mvp_winners = {
    1956: "Bob Pettit",
    1957: "Bob Cousy",
    1958: "Bill Russell",
    1959: "Bob Pettit",
    1960: "Wilt Chamberlain",
    1961: "Bill Russell",
    1962: "Bill Russell",
    1963: "Bill Russell",
    1964: "Oscar Robertson",
    1965: "Bill Russell",
    1966: "Wilt Chamberlain",
    1967: "Wilt Chamberlain",
    1968: "Wilt Chamberlain",
    1969: "Wes Unseld",
    1970: "Willis Reed",
    1971: "Kareem Abdul-Jabbar",
    1972: "Kareem Abdul-Jabbar",
    1973: "Dave Cowens",
    1974: "Kareem Abdul-Jabbar",
    1975: "Bob McAdoo",
    1976: "Kareem Abdul-Jabbar",
    1977: "Kareem Abdul-Jabbar",
    1978: "Bill Walton",
    1979: "Moses Malone",
    1980: "Kareem Abdul-Jabbar",
    1981: "Julius Erving",
    1982: "Moses Malone",
    1983: "Moses Malone",
    1984: "Larry Bird",
    1985: "Larry Bird",
    1986: "Larry Bird",
    1987: "Magic Johnson",
    1988: "Michael Jordan",
    1989: "Magic Johnson",
    1990: "Magic Johnson",
    1991: "Michael Jordan",
    1992: "Michael Jordan",
    1993: "Charles Barkley",
    1994: "Hakeem Olajuwon",
    1995: "David Robinson",
    1996: "Michael Jordan",
    1997: "Karl Malone",
    1998: "Michael Jordan",
    1999: "Karl Malone",
    2000: "Shaquille O'Neal",
    2001: "Allen Iverson",
    2002: "Tim Duncan",
    2003: "Tim Duncan",
    2004: "Kevin Garnett",
    2005: "Steve Nash",
    2006: "Steve Nash",
    2007: "Dirk Nowitzki",
    2008: "Kobe Bryant",
    2009: "LeBron James",
    2010: "LeBron James",
    2011: "Derrick Rose",
    2012: "LeBron James",
    2013: "LeBron James",
    2014: "Kevin Durant",
    2015: "Stephen Curry",
    2016: "Stephen Curry",
    2017: "Russell Westbrook",
    2018: "James Harden"
}

In [9]:
import random
random.seed(42)

years = list(mvp_winners.keys())
random.shuffle(years)
split_index = int(0.8 * len(years))
train_years = years[:split_index]
test_years = years[split_index:]

train_mvp_winners = {year: mvp_winners[year] for year in train_years}
test_mvp_winners = {year: mvp_winners[year] for year in test_years}

In [10]:
print(train_mvp_winners)
print(test_mvp_winners)

{1992: 'Michael Jordan', 1977: 'Kareem Abdul-Jabbar', 1997: 'Karl Malone', 1989: 'Magic Johnson', 1975: 'Bob McAdoo', 2018: 'James Harden', 2013: 'LeBron James', 2007: 'Dirk Nowitzki', 1995: 'David Robinson', 1986: 'Larry Bird', 2017: 'Russell Westbrook', 1987: 'Magic Johnson', 1974: 'Kareem Abdul-Jabbar', 1979: 'Moses Malone', 1965: 'Bill Russell', 1972: 'Kareem Abdul-Jabbar', 1967: 'Wilt Chamberlain', 1976: 'Kareem Abdul-Jabbar', 2004: 'Kevin Garnett', 2002: 'Tim Duncan', 1959: 'Bob Pettit', 1985: 'Larry Bird', 2009: 'LeBron James', 1960: 'Wilt Chamberlain', 2011: 'Derrick Rose', 1966: 'Wilt Chamberlain', 2008: 'Kobe Bryant', 1978: 'Bill Walton', 2000: "Shaquille O'Neal", 1981: 'Julius Erving', 1980: 'Kareem Abdul-Jabbar', 1956: 'Bob Pettit', 2014: 'Kevin Durant', 1984: 'Larry Bird', 1998: 'Michael Jordan', 1982: 'Moses Malone', 2006: 'Steve Nash', 1968: 'Wilt Chamberlain', 1991: 'Michael Jordan', 2001: 'Allen Iverson', 1994: 'Hakeem Olajuwon', 1988: 'Michael Jordan', 2012: 'LeBron J

In [11]:
df['Won_MVP'] = df.apply(lambda row: int(row['Player'] == mvp_winners.get(row['Year'])), axis=1)
df.head()

,Year,Player,Pos,Age,Tm,G,GS,MP,PER,TS%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Won_MVP
3851,1974,Zaid Abdul-Aziz,C,27.0,HOU,79.0,NaN,2459.0,15.9,0.516,...,259.0,664.0,923.0,166.0,80.0,104.0,NaN,227.0,865.0,0
3852,1974,Kareem Abdul-Jabbar,C,26.0,MIL,81.0,NaN,3548.0,24.4,0.564,...,287.0,891.0,1178.0,386.0,112.0,283.0,NaN,238.0,2191.0,1
3853,1974,Don Adams,SF,26.0,DET,74.0,NaN,2298.0,10.9,0.457,...,133.0,315.0,448.0,141.0,110.0,12.0,NaN,242.0,759.0,0
3854,1974,Rick Adelman,PG,27.0,CHI,55.0,NaN,618.0,10.0,0.447,...,16.0,53.0,69.0,56.0,36.0,1.0,NaN,63.0,182.0,0
3855,1974,Lucius Allen,PG,26.0,MIL,72.0,NaN,2388.0,18.8,0.536,...,89.0,202.0,291.0,374.0,137.0,22.0,NaN,215.0,1268.0,0


In [12]:
print(df[df['Won_MVP'] == 1])

       Year               Player Pos   Age   Tm     G    GS      MP   PER  \
3852   1974  Kareem Abdul-Jabbar   C  26.0  MIL  81.0   NaN  3548.0  24.4   
4255   1975           Bob McAdoo   C  23.0  BUF  82.0   NaN  3539.0  25.8   
4375   1976  Kareem Abdul-Jabbar   C  28.0  LAL  82.0   NaN  3379.0  27.2   
4650   1977  Kareem Abdul-Jabbar   C  29.0  LAL  82.0   NaN  3016.0  27.8   
5345   1978          Bill Walton   C  25.0  POR  58.0   NaN  1929.0  24.8   
5580   1979         Moses Malone   C  23.0  HOU  82.0   NaN  3390.0  23.7   
5727   1980  Kareem Abdul-Jabbar   C  32.0  LAL  82.0   NaN  3143.0  25.3   
6179   1981        Julius Erving  SF  30.0  PHI  82.0   NaN  2874.0  25.1   
6672   1982         Moses Malone   C  26.0  HOU  81.0  81.0  3398.0  26.8   
7045   1983         Moses Malone   C  27.0  PHI  78.0  78.0  2922.0  25.1   
7230   1984           Larry Bird  PF  27.0  BOS  79.0  77.0  3028.0  24.2   
7574   1985           Larry Bird  SF  28.0  BOS  80.0  77.0  3161.0  26.5   

In [13]:
print(list(df.columns))

['Year', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Won_MVP']


In [21]:
from pandas.api.types import is_numeric_dtype, is_object_dtype

# -------------------- Preprocessing --------------------

def load_and_preprocess_from_df(df):
    df = df.copy()
    df = df.dropna(subset=['Won_MVP'])

    y = df['Won_MVP'].astype(int).values

    exclude_cols = ['Player', 'Tm', 'Won_MVP', 'Year']
    features = [col for col in df.columns if col not in exclude_cols]

    # Fill NaNs with median for numerical columns
    for col in features:
        if df[col].isnull().any():
            median = df[col].median()
            df[col] = df[col].fillna(median)

    # Bin continuous variables into integers (5 bins per feature)
    bins_dict = {}
    X = []
    for col in features:
        if is_object_dtype(df[col].dtype):
            continue
        try:
            binned, bins = pd.qcut(df[col], q=5, labels=False, retbins=True, duplicates='drop')
            if binned.isnull().any():
                continue  # Skip if qcut failed
            X.append(binned.astype(int).values)
            bins_dict[col] = bins
        except ValueError:
            continue  # Skip if qcut fails due to not enough unique values

    X = np.stack(X, axis=1)
    return X, y, features, bins_dict

# -------------------- Pyro Naive Bayes Model --------------------

def build_model(feature_bins, num_classes=2):
    def model(X, y=None):
        class_probs = pyro.param("class_probs", torch.ones(num_classes) / num_classes,
                                 constraint=dist.constraints.simplex)

        with pyro.plate("data", X.shape[0]):
            y_hat = pyro.sample("y", dist.Categorical(class_probs), obs=y)
            for i, bins in enumerate(feature_bins):
                probs = pyro.param(f"feature_{i}_probs", torch.ones(num_classes, bins) / bins,
                                   constraint=dist.constraints.simplex)
                pyro.sample(f"feature_{i}", dist.Categorical(probs[y_hat]), obs=X[:, i])
    return model

def guide(X, y=None):
    pass  # No guide needed for Naive Bayes

# -------------------- Training --------------------

def train(X, y, feature_bins):
    pyro.clear_param_store()

    model = build_model(feature_bins)
    svi = SVI(model, guide, Adam({"lr": 0.01}), loss=Trace_ELBO())

    X_tensor = torch.tensor(X, dtype=torch.long)
    y_tensor = torch.tensor(y, dtype=torch.long)

    for step in range(100):
        loss = svi.step(X_tensor, y_tensor)
        if step % 50 == 0:
            print(f"[{step}] Loss: {loss:.2f}")

# -------------------- Prediction --------------------

def predict(X_new, feature_bins):
    class_probs = pyro.param("class_probs").detach().numpy()
    log_probs = np.log(class_probs)

    preds = []
    for row in X_new:
        logp = np.log(class_probs.copy())
        for i, val in enumerate(row):
            probs = pyro.param(f"feature_{i}_probs").detach().numpy()
            logp += np.log(probs[:, val])
        preds.append(np.argmax(logp))
    return np.array(preds)

# def predict_proba(X_new, feature_bins):
#     class_probs = pyro.param("class_probs").detach().numpy()
#     preds_proba = []
#     for row in X_new:
#         logp = np.log(class_probs.copy())
#         for i, val in enumerate(row):
#             probs = pyro.param(f"feature_{i}_probs").detach().numpy()
#             logp += np.log(probs[:, val])
#         proba = np.exp(logp - logp.max())  # For numerical stability
#         proba = proba / proba.sum()
#         preds_proba.append(proba[1])  # Probability of being MVP
#     return np.array(preds_proba)

# -------------------- Main Execution --------------------

# def run_pipeline_from_df(df):
#     X, y, features, bins_dict = load_and_preprocess_from_df(df)
#     feature_bins = [len(bins_dict[col]) - 1 for col in features if is_numeric_dtype(df[col])]

#     print("Training model...")
#     train(X, y, feature_bins)

#     print("Evaluating on training data...")

#     # Predict MVP probabilities
#     def predict_proba(X_new, feature_bins):
#         class_probs = pyro.param("class_probs").detach().numpy()
#         preds_proba = []
#         for row in X_new:
#             logp = np.log(class_probs.copy())
#             for i, val in enumerate(row):
#                 probs = pyro.param(f"feature_{i}_probs").detach().numpy()
#                 logp += np.log(probs[:, val])
#             proba = np.exp(logp - logp.max())  # For numerical stability
#             proba = proba / proba.sum()
#             preds_proba.append(proba[1])  # Probability of being MVP
#         return np.array(preds_proba)

#     probs = predict_proba(X, feature_bins)

#     df_eval = df.copy()
#     df_eval["MVP_Prob"] = probs
#     df_eval["Actual_MVP"] = y

#     # Select top player per year
#     top_preds = df_eval.loc[df_eval.groupby("Year")["MVP_Prob"].idxmax()]
#     df_eval["Predicted_MVP"] = 0
#     df_eval.loc[top_preds.index, "Predicted_MVP"] = 1

#     # Compute accuracy by year
#     correct_by_year = (top_preds["Actual_MVP"] == 1).sum()
#     total_years = top_preds["Year"].nunique()
#     print(f"Correct MVP Predictions by Year: {correct_by_year}/{total_years} ({100 * correct_by_year / total_years:.2f}%)")

#     # Print relevant rows
#     print("\nPredictions on training data:")
#     print(df_eval[(df_eval["Predicted_MVP"] == 1) | (df_eval["Actual_MVP"] == 1)][
#         ["Player", "Year", "Predicted_MVP", "Actual_MVP", "MVP_Prob"]
#     ])

def run_per_year_pipeline(df, years, mvp_dict):
    correct = 0
    total = 0
    results = []

    for year in years:
        df_year = df[df['Year'] == year].copy()
        if df_year.empty or year not in mvp_dict:
            continue

        mvp = mvp_dict[year]
        print(f"Processing year {year}")
        print(df_year['Won_MVP'].isna().sum())

        df_year['Won_MVP'] = df_year['Player'].apply(lambda p: int(p.strip() == mvp.strip()))

        X, y, features, bins_dict = load_and_preprocess_from_df(df_year)
        feature_bins = [len(bins_dict[col]) - 1 for col in features if col in bins_dict]

        train(X, y, feature_bins)

        def predict_proba(X_new, feature_bins):
            class_probs = pyro.param("class_probs").detach().numpy()
            preds_proba = []
            for row in X_new:
                logp = np.log(class_probs.copy())
                for i, val in enumerate(row):
                    probs = pyro.param(f"feature_{i}_probs").detach().numpy()
                    val = min(val, probs.shape[1] - 1)  # ensure valid index
                    logp += np.log(probs[:, val])
                proba = np.exp(logp - logp.max())  # For numerical stability
                proba = proba / proba.sum()
                preds_proba.append(proba[1])  # Probability of being MVP
            return np.array(preds_proba)

        probs = predict_proba(X, feature_bins)

        df_year["MVP_Prob"] = probs
        df_year["Actual_MVP"] = y

        top_pred_idx = df_year["MVP_Prob"].idxmax()
        predicted_mvp = df_year.loc[top_pred_idx]

        correct += int(predicted_mvp["Actual_MVP"] == 1)
        total += 1

        results.append(predicted_mvp[["Player", "Year", "MVP_Prob", "Actual_MVP"]])

    print(f"\nCorrect MVP Predictions: {correct}/{total} ({100 * correct / total:.2f}%)")
    return pd.DataFrame(results)

train_results = run_per_year_pipeline(df, train_years, train_mvp_winners)
test_results = run_per_year_pipeline(df, test_years, test_mvp_winners)

print(train_results)
print(test_results)

Processing year 1992
0
[0] Loss: 33786.74
[50] Loss: 33401.20
Processing year 1977
0
[0] Loss: 22081.09
[50] Loss: 21890.61
Processing year 1997
0
[0] Loss: 42509.21
[50] Loss: 42109.96
Processing year 1989
0
[0] Loss: 32185.33
[50] Loss: 31758.59
Processing year 1975
0
[0] Loss: 17071.09
[50] Loss: 16905.29
Processing year 2013
0
[0] Loss: 42563.01
[50] Loss: 42176.33
Processing year 2007
0
[0] Loss: 38329.00
[50] Loss: 37979.16


KeyboardInterrupt: 